In [45]:
import pandas as pd
import matplotlib.pyplot as plt

Lectura de archivos

In [46]:
tweets_df = pd.read_csv("../data/tweets_with_sentiment_cryptobert.csv")
tweets_df['Created At'] = pd.to_datetime(tweets_df['Created At'], errors='coerce')
tweets_df['fecha'] = tweets_df['Created At'].dt.date

price_df = pd.read_csv("../data/datos_mercado_binance.csv")
price_df['fecha'] = pd.to_datetime(price_df['fecha'], errors='coerce')

In [47]:
# We count by kind of feeling
conteo_sentimientos = tweets_df.groupby(['fecha', 'Sentiment']).size().unstack(fill_value=0)

# We recalculate totals and average
conteo_sentimientos['numero_noticias'] = conteo_sentimientos.sum(axis=1)
conteo_sentimientos['promedio_sentimiento'] = tweets_df.groupby('fecha')['Sentiment'].mean()

In [48]:
# Ensure that both 'date' columns are of type datetime.date
price_df['fecha'] = pd.to_datetime(price_df['fecha']).dt.date
conteo_sentimientos = conteo_sentimientos.reset_index()  # si 'fecha' está en el índice
conteo_sentimientos['fecha'] = pd.to_datetime(conteo_sentimientos['fecha']).dt.date

# We join by date
df_final = price_df.merge(conteo_sentimientos, on='fecha', how='left')

# We added trend column
window = 10
df_final['media_movil_10d'] = df_final['precio_apertura'].rolling(window).mean()
df_final['derivada_mm10'] = df_final['media_movil_10d'].diff()
def clasificar_derivada(valor):
    if pd.isna(valor):
        return 0  
    elif valor > 0:
        return 1
    elif valor < 0:
        return -1
    else:
        return 0
df_final['tendencia_categorizada'] = df_final['derivada_mm10'].apply(clasificar_derivada)


In [49]:
# Selecting final columns
df_csv = df_final[['fecha', 'precio_apertura', 'tendencia_categorizada', 'numero_noticias', -1, 0, 1, 'promedio_sentimiento']]

# We rename columns
df_csv.columns = ['DATE', 'OPEN', 'TREND', 'NEWS ISSUE', 'NEGATIVE', 'NEUTRAL', 'POSITIVE', 'AVERAGE NEWS']

# We save CSV
df_csv.to_csv('../data/analisis_completo.csv', index=False)